## SA - Lenta

In [3]:
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
import torch
from torch import cuda

In [19]:
# Определение устройства для работы с CUDA, если доступно
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [20]:
# Загрузка данных из CSV-файла
data = pd.read_csv('Train_Lenta.csv', encoding='utf-8-sig', sep=';')
data.head()

,id,text
0,800000,Самый низкий показатель - среди жителей Японии...
1,800001,В самих США положительно ответили более 20 про...
2,800002,"Аналитики MPAA считают, что распространение пи..."
3,800003,При недостаточно высокой скорости соединения с...
4,800004,Лидерство Южной Кореи в опросе обусловлено име...


In [21]:
# Замена пропущенных значений или значений других типов на пустые строки
data['text'] = data['text'].fillna('').astype(str)

In [22]:
# Загрузка модели для sentiment analysis
classifier = pipeline("sentiment-analysis", model='seara/rubert-tiny2-russian-sentiment')

# Создание списка текстов для анализа
texts = data['text'].tolist()

# Применение модели к данным с отслеживанием прогресса
sentiments = []
for text in tqdm(texts, desc="Analyzing sentiment"):
    sentiment = classifier(text)
    sentiments.append(sentiment)

# Добавление результатов анализа в новую колонку 'sentiment'
data['sentiment'] = [sent[0]['label'] for sent in sentiments]

Analyzing sentiment: 100%|██████████| 1009468/1009468 [39:52<00:00, 421.93it/s]


In [23]:
# Сохранение обновленных данных в новый CSV-файл
data.to_csv('Train_Lenta_SA.csv', encoding='utf-8-sig', sep=';', index=False)

In [24]:
data = pd.read_csv('Train_Lenta_SA.csv', encoding='utf-8-sig', sep=';')
data.head()

,id,text,sentiment
0,800000,Самый низкий показатель - среди жителей Японии...,neutral
1,800001,В самих США положительно ответили более 20 про...,neutral
2,800002,"Аналитики MPAA считают, что распространение пи...",neutral
3,800003,При недостаточно высокой скорости соединения с...,neutral
4,800004,Лидерство Южной Кореи в опросе обусловлено име...,neutral


In [25]:
# Подсчет количества уникальных значений в колонке 'label'
label_counts = data['sentiment'].value_counts()

# Вывод результата
print(label_counts)

neutral     880165
negative    100393
positive     28910
Name: sentiment, dtype: int64


## SA - Reviews

In [26]:
# Загрузка данных из CSV-файла
data = pd.read_csv('Reviews_Train.csv', encoding='utf-8-sig', sep=';')
data.head()

,id,text
0,1,Не дорого!
1,2,Не дорого.
2,3,Не жалеем!
3,4,Не жалеем.
4,5,Не за что!


In [27]:
# Замена пропущенных значений или значений других типов на пустые строки
data['text'] = data['text'].fillna('').astype(str)

In [28]:
# Создание списка текстов для анализа
texts = data['text'].tolist()

# Применение модели к данным с отслеживанием прогресса
sentiments = []
for text in tqdm(texts, desc="Analyzing sentiment"):
    sentiment = classifier(text)
    sentiments.append(sentiment)

# Добавление результатов анализа в новую колонку 'sentiment'
data['sentiment'] = [sent[0]['label'] for sent in sentiments]

Analyzing sentiment: 100%|██████████| 457575/457575 [16:15<00:00, 469.25it/s]


In [29]:
# Сохранение обновленных данных в новый CSV-файл
data.to_csv('Reviews_Train_SA.csv', encoding='utf-8-sig', sep=';', index=False)

In [30]:
data = pd.read_csv('Reviews_Train_SA.csv', encoding='utf-8-sig', sep=';')
data.head()

,id,text,sentiment
0,1,Не дорого!,neutral
1,2,Не дорого.,neutral
2,3,Не жалеем!,neutral
3,4,Не жалеем.,neutral
4,5,Не за что!,neutral


In [31]:
# Подсчет количества уникальных значений в колонке 'label'
label_counts = data['sentiment'].value_counts()

# Вывод результата
print(label_counts)

neutral     273279
positive    167813
negative     16483
Name: sentiment, dtype: int64


## NER - Reviews

In [35]:
from deeppavlov import configs, build_model

config_path = configs.ner.ner_rus_bert

ner = build_model(config_path, download=True)

2024-04-09 02:27:56.618 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz download because of matching hashes
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you ex

In [36]:
data = pd.read_csv('Reviews_Train.csv', encoding='utf-8-sig', sep=';')
data.head()

,id,text
0,1,Не дорого!
1,2,Не дорого.
2,3,Не жалеем!
3,4,Не жалеем.
4,5,Не за что!


In [37]:
# Замена пропущенных значений или значений других типов на пустые строки
data['text'] = data['text'].fillna('').astype(str)

In [38]:
# Очистка текста от лишних символов
data['text'] = data['text'].str.strip()

# Применение модели NER к текстам из колонки 'text' и сохранение результатов в новую колонку 'tag'
tags_list = []
for sentence in tqdm(data['text'], desc="Processing sentences"):
    if isinstance(sentence, str) and sentence.strip():  # Проверяем, является ли значение текстом и не пустым
        tokens, tags = ner([sentence])
        tags_list.append(tags[0])
    else:
        tags_list.append([])  # Добавляем пустой список тегов

data['tag'] = tags_list

Processing sentences: 100%|██████████| 457575/457575 [1:09:00<00:00, 110.52it/s]


In [39]:
# Сохранение обновленных данных в новый CSV-файл
data.to_csv('Reviews_Train_NER.csv', encoding='utf-8-sig', sep=';', index=False)

In [40]:
data = pd.read_csv('Reviews_Train_NER.csv', encoding='utf-8-sig', sep=';')
data.head()

,id,text,tag
0,1,Не дорого!,"['O', 'O', 'O']"
1,2,Не дорого.,"['O', 'O', 'O']"
2,3,Не жалеем!,"['O', 'O', 'O']"
3,4,Не жалеем.,"['O', 'O', 'O']"
4,5,Не за что!,"['O', 'O', 'O', 'O']"
